# Code Assistant

In [ ]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import ollama
import gradio as gr
from google import genai
from google.genai import types

In [ ]:
# load environment variables

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

In [ ]:
# connect to openai, google, anthropic

openai = OpenAI()
claude = anthropic.Anthropic()
gemmini = genai.Client()

In [ ]:
# ensure ollama is loaded 

!ollama pull llama3.2

In [ ]:
# define Models

MODEL_GPT = 'gpt-4o-mini'
MODEL_OLLAMA = 'llama3.2'
MODEL_CLAUDE = 'claude-3-haiku-20240307'
MODEL_GEMINI = 'gemini-2.0-flash'

In [ ]:
# define system prompt

system_prompt = '''
    You are AI assistant that helps explain what code does and why.
    You should respond in Markdown.
'''

In [ ]:
# gpt call

def message_gpt(question):
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': question}
    ]
    stream = openai.chat.completions.create(
        messages=messages,
        model=MODEL_GPT, 
        max_tokens=1000,
        stream=True
    )

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result
    

In [ ]:
# ollama call

def message_ollama(question):
    messages = [
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': question}
    ]
    stream = ollama.chat(
        model=MODEL_OLLAMA,
        messages=messages,
        stream=True
    )

    result = ''
    for chunk in stream:
        result += chunk['message']['content'] or ''
        yield result

In [ ]:
# gemini call

def message_gemmini(question):
    stream = gemmini.models.generate_content_stream(
        model=MODEL_GEMINI,
        config=types.GenerateContentConfig(system_instruction=system_prompt),
        contents=[question]
    )
    result = ''
    for chunk in stream:
        result += chunk.text or ''
        yield result


In [ ]:
# claude call

def message_claude(question):
    messages = [{'role': 'user', 'content': question}]
    result = claude.messages.stream(
        model=MODEL_CLAUDE,
        max_tokens=1000,
        system=system_prompt,
        messages=messages
    )
    response = ""
    with result as stream:
        for text in stream.text_stream:
            response += text or ""
            yield response

In [ ]:
def stream_model(model, prompt):

    if model == 'GPT':
        result = message_gpt(prompt)
    elif model == 'Claude':
        result = message_claude(prompt)
    elif model == 'Gemmini':
        result = message_gemmini(prompt)
    elif model == 'llama':
        message_ollama(prompt)
    else:
        raise ValueError("Unknown model")

    

    yield from result

In [ ]:
# build UI with Gradio

view = gr.Interface(
    fn=stream_model,
    inputs=[
        gr.Dropdown(['GPT', 'Claude', 'Gemmini', 'llama'], label='Select Model:', value='GPT'),
        gr.Textbox(label="Question:")
    ],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()